# ETL
* EXTRACT 
* TRANSFORM
* LOAD 

* En este notebook se realiza un ETL de los datasets para realizar nuestro proyecto, brindandoles la estructura necesaria para realizar un modelo de Machine Learning 

In [27]:
# Cargamos las librerias que utilizaremos para nuestro ETL 

import pandas as pd
import gzip
import datetime as dt
import ast
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

#### INGESTA DE DATOS (EXTRACT)

In [28]:
# Abre el archivo 'steam_games.json.gz' con codificación UTF-8 utilizando gzip.
# 'rt' significa "modo de texto para lectura".
with gzip.open('C:/Users/hp/Desktop/HENRY/Soy henry Modulos/PI ML/PI MLOps - STEAM/Datasets/steam_games.json.gz', 'rt', encoding='utf-8') as steam_games:
    
    # Lee el archivo JSON línea por línea y crea un DataFrame.
    # El argumento 'lines=True' indica que cada línea es un objeto JSON separado.
    df_steam_games = pd.read_json(steam_games, lines=True)

# Muestra el DataFrame resultante.
df_steam_games  

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [29]:
# creamos una funcion para abrir el archivo "users_reviews.json.gz"
def cargar_user_reviews_desde_gzip(archivo):
    """
    Carga datos de un archivo comprimido en formato Python (no JSON) y los almacena en un DataFrame.

    Args:
        archivo (str): Ruta al archivo comprimido que contiene los datos.

    Returns:
        pd.DataFrame: Un DataFrame que contiene los datos cargados desde el archivo.
    """
    datos = []

    # Abre el archivo comprimido y recorre las líneas.
    for i in gzip.open(archivo):
        # Decodifica cada línea y utiliza 'ast.literal_eval' para evaluar el contenido como un objeto Python.
        # El resultado se agrega a la lista 'datos'.
        datos.append(ast.literal_eval(i.decode('utf-8')))

    # Crea un DataFrame de Pandas a partir de la lista 'datos'.
    df = pd.DataFrame(datos)

    return df



In [30]:
# Para utilizar la función y cargar los datos desde el archivo:
df_user_reviews = cargar_user_reviews_desde_gzip('C:/Users/hp/Desktop/HENRY/Soy henry Modulos/PI ML/PI MLOps - STEAM/Datasets/user_reviews.json.gz')
df_user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [31]:
# Crea una lista llamada 'datos_items' para almacenar los datos extraídos del archivo JSON.
datos_items = []

# Abre el archivo comprimido 'users_items.json.gz' con gzip y lo recorre línea por línea.
for i in gzip.open('C:/Users/hp/Desktop/HENRY/Soy henry Modulos/PI ML/PI MLOps - STEAM/Datasets/users_items.json.gz'):
    # Decodifica cada línea del archivo y luego utiliza 'ast.literal_eval' para evaluar el contenido como un objeto Python.
    # El resultado se agrega a la lista 'datos_items'.
    datos_items.append(ast.literal_eval(i.decode('utf-8')))

# Crea un DataFrame de Pandas a partir de la lista 'datos_items', lo que permite organizar los datos en una estructura tabular.
df_users_items = pd.DataFrame(datos_items)

# df_users_items ahora contiene los datos del archivo 'users_items.json.gz'
# Muestra el DataFrame resultante 
df_users_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


##### RESUMEN
        * se descomprimieron los datasets de formato .json.gzip
        * se introjuerdon a un DataFrame de Pandas para porder trabajar con ellos 

#### TRATAMIENTO DE DATOS (TRANSFORM)

<H1 style="font-size: 20px; color: gold;"> Dataset steam_games </H1>

In [32]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [33]:

# se transfroma la columna release_date de object a datetime 
df_steam_games['release_date'] = pd.to_datetime(df_steam_games['release_date'], errors='coerce')
# Cambia el formato de las fechas a 'AAAA-MM-DD'
df_steam_games['release_date'] = df_steam_games['release_date'].dt.strftime('%Y-%m-%d')
# se transforma nuevamente la columna ya que al cmbiar el formato de la fecha se transforma en object 
df_steam_games['release_date'] = pd.to_datetime(df_steam_games['release_date'], errors='coerce')
# verificamos que las fechas esten en el formato correcto
df_steam_games['release_date'].head()

0   NaT
1   NaT
2   NaT
3   NaT
4   NaT
Name: release_date, dtype: datetime64[ns]

In [34]:
# eliminamos las columnas que no son necesarias para nuestro analisis 
df_steam_games.drop(columns=['publisher','specs','price','early_access','developer'], axis=1, inplace=True)

In [35]:
# contamos la cantidad de datos faltantes en cada columna 
df_steam_games.isnull().sum()

genres          91593
app_name        88312
title           90360
url             88310
release_date    90662
tags            88473
reviews_url     88312
id              88312
dtype: int64

In [36]:
# convertimos las listas de el dataframe en cadenas(string)
df_steam_games = df_steam_games.applymap(lambda x: str(x) if isinstance(x, list) else x)

In [37]:
# verificamos si existen datos duplicados en nuestro dataset
df_steam_games.duplicated().sum()

88309

In [38]:
# eliminamos losdatos nulos ya que algunas funciones no nos permiten trabajar con datos nulos o faltantes 
df_steam_games['app_name'] = df_steam_games['app_name'].dropna(inplace = True)

In [39]:
# reseteamos el index para que empiece la cuenta en 0 
df_steam_games.reset_index(inplace=True,drop=True)
# verficamos que se ejecute correctamente
df_steam_games.tail()


,genres,app_name,title,url,release_date,tags,reviews_url,id
120440,"['Casual', 'Indie', 'Simulation', 'Strategy']",None,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"['Strategy', 'Indie', 'Casual', 'Simulation']",http://steamcommunity.com/app/773640/reviews/?...,773640.0
120441,"['Casual', 'Indie', 'Strategy']",None,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"['Strategy', 'Indie', 'Casual']",http://steamcommunity.com/app/733530/reviews/?...,733530.0
120442,"['Indie', 'Racing', 'Simulation']",None,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"['Indie', 'Simulation', 'Racing']",http://steamcommunity.com/app/610660/reviews/?...,610660.0
120443,"['Casual', 'Indie']",None,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",http://steamcommunity.com/app/658870/reviews/?...,658870.0
120444,None,None,None,http://store.steampowered.com/app/681550/Maze_...,NaT,"['Early Access', 'Adventure', 'Indie', 'Action...",http://steamcommunity.com/app/681550/reviews/?...,681550.0


In [40]:
#eliminamos los datos nulos
df_steam_games.dropna()


,genres,app_name,title,url,release_date,tags,reviews_url,id


In [41]:
# revisamos por ultima vez que el dataset este limpio y listo para su uso 
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   genres        28852 non-null  object        
 1   app_name      0 non-null      object        
 2   title         30085 non-null  object        
 3   url           32135 non-null  object        
 4   release_date  29783 non-null  datetime64[ns]
 5   tags          31972 non-null  object        
 6   reviews_url   32133 non-null  object        
 7   id            32133 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 7.4+ MB


In [42]:
df_steam_games

,genres,app_name,title,url,release_date,tags,reviews_url,id
0,None,None,None,None,NaT,None,None,NaN
1,None,None,None,None,NaT,None,None,NaN
2,None,None,None,None,NaT,None,None,NaN
3,None,None,None,None,NaT,None,None,NaN
4,None,None,None,None,NaT,None,None,NaN
...,...,...,...,...,...,...,...,...
120440,"['Casual', 'Indie', 'Simulation', 'Strategy']",None,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"['Strategy', 'Indie', 'Casual', 'Simulation']",http://steamcommunity.com/app/773640/reviews/?...,773640.0
120441,"['Casual', 'Indie', 'Strategy']",None,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"['Strategy', 'Indie', 'Casual']",http://steamcommunity.com/app/733530/reviews/?...,733530.0
120442,"['Indie', 'Racing', 'Simulation']",None,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"['Indie', 'Simulation', 'Racing']",http://steamcommunity.com/app/610660/reviews/?...,610660.0
120443,"['Casual', 'Indie']",None,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",http://steamcommunity.com/app/658870/reviews/?...,658870.0


<H1 style="font-size: 20px; color: gold;"> Dataset user_reviews </H1>

In [43]:
df_user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [44]:
# Expandir(desanidar) la columna 'reviews' del DataFrame df_user_reviews
df_reviews_exploded = df_user_reviews.explode('reviews')

# Normalizar los datos anidados en 'reviews' y desanidarlos en un nuevo DataFrame
df_reviews_desanidado = pd.json_normalize(df_reviews_exploded['reviews'].dropna())

# Reindexar los DataFrames
df_reviews_desanidado.reset_index(inplace=True)
df_reviews_exploded.reset_index(inplace=True)

# Concatenar los DataFrames y eliminar la columna 'reviews' que originalmente desanidamos
user_reviews = pd.concat([df_reviews_exploded, df_reviews_desanidado], axis=1)
df_user_reviews = user_reviews.drop(columns=['reviews'])

# Mostrar el DataFrame resultante
df_user_reviews

,index,user_id,user_url,index,funny,posted,last_edited,item_id,helpful,recommend,review
0,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0.0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1.0,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2.0,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,1,js41637,http://steamcommunity.com/id/js41637,3.0,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,1,js41637,http://steamcommunity.com/id/js41637,4.0,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...,...,...
59328,25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59329,25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59330,25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59331,25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        59333 non-null  int64  
 1   user_id      59333 non-null  object 
 2   user_url     59333 non-null  object 
 3   index        59305 non-null  float64
 4   funny        59305 non-null  object 
 5   posted       59305 non-null  object 
 6   last_edited  59305 non-null  object 
 7   item_id      59305 non-null  object 
 8   helpful      59305 non-null  object 
 9   recommend    59305 non-null  object 
 10  review       59305 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 5.0+ MB


In [46]:
# eliminamos las columnas que no son necesarias 
df_user_reviews.drop(columns=['index','funny','helpful','index', 'last_edited'], axis=1, inplace=True)

In [47]:
# contamos la cantidad de nulos en el dataset
df_user_reviews.isnull().sum()

user_id       0
user_url      0
posted       28
item_id      28
recommend    28
review       28
dtype: int64

In [48]:
df_user_reviews.dropna(inplace=True)

In [49]:

df_user_reviews.duplicated().sum()

144

In [50]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59305 entries, 0 to 59304
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59305 non-null  object
 1   user_url   59305 non-null  object
 2   posted     59305 non-null  object
 3   item_id    59305 non-null  object
 4   recommend  59305 non-null  object
 5   review     59305 non-null  object
dtypes: object(6)
memory usage: 3.2+ MB


In [51]:
def convertir_string_fecha(date_string):
    # Dividir la cadena en palabras
    words = date_string.split()
    
    if len(words) < 4:
        # Si no hay suficientes palabras en la cadena, devuelve una fecha nula
        return pd.NaT

    month = words[1]
    day = int(words[2].rstrip(','))
    year = int(words[3].rstrip('.'))
    
    date_format = f"{year}, {month}, {day}"
    
    try:
        return pd.to_datetime(date_format, format='%Y, %B, %d')
    except ValueError:
        # Si la conversión falla, devuelve una fecha nula
        return pd.NaT

# Aplicar la función a la columna completa
df_user_reviews['posted'] = df_user_reviews['posted'].apply(convertir_string_fecha)



In [52]:
# transforma el tipo de dato de la columna 'item_id' 
df_user_reviews['item_id']= df_user_reviews['item_id'].astype(int)

In [53]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59305 entries, 0 to 59304
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   user_id    59305 non-null  object        
 1   user_url   59305 non-null  object        
 2   posted     49186 non-null  datetime64[ns]
 3   item_id    59305 non-null  int32         
 4   recommend  59305 non-null  object        
 5   review     59305 non-null  object        
dtypes: datetime64[ns](1), int32(1), object(4)
memory usage: 2.9+ MB


In [54]:
# creamos la columna sentiment_analysis
# Crear un analizador de sentimientos VADER
sia = SentimentIntensityAnalyzer()

# Función para asignar valores de sentimiento
def analisis_de_sentimiento(texto):
    if pd.isna(texto):
        # Si la reseña está ausente, asignar valor neutral '1'
        return 1
    else:
        # Obtener el puntaje de sentimiento
        score = sia.polarity_scores(texto)
        # Clasificar el sentimiento basado en la puntuación compuesta
        if score['compound'] > 0.05:
            return 2  # Sentimiento positivo
        elif score['compound'] < -0.05:
            return 0  # Sentimiento negativo
        else:
            return 1  # Sentimiento neutral

# Aplicar la función a la columna de reseñas y crear la nueva columna 'sentiment_analysis'
df_user_reviews['sentiment_analysis'] = df_user_reviews['review'].apply(analisis_de_sentimiento)

# Reemplazar la columna 'review' por 'sentiment_analysis' si es necesario
df_user_reviews.drop('review', axis=1, inplace=True)


In [55]:
df_user_reviews

,user_id,user_url,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-11-05,1250,True,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-07-15,22200,True,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2011-04-21,43110,True,2
3,js41637,http://steamcommunity.com/id/js41637,2014-06-24,251610,True,2
4,js41637,http://steamcommunity.com/id/js41637,2013-09-08,227300,True,2
...,...,...,...,...,...,...
59300,Fuckfhaisjnsnsjakaka,http://steamcommunity.com/id/Fuckfhaisjnsnsjakaka,NaT,70,True,2
59301,3214213216,http://steamcommunity.com/id/3214213216,NaT,362890,True,2
59302,ChrisCoroner,http://steamcommunity.com/id/ChrisCoroner,NaT,273110,True,2
59303,CaptainAmericaCw,http://steamcommunity.com/id/CaptainAmericaCw,NaT,730,True,2


<H1 style="font-size: 20px; color: gold;"> Dataset users_items </H1>

In [56]:
df_users_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [57]:
# Explodir la columna 'items' del DataFrame df_users_items
df_items_exploded = df_users_items.explode('items')

# Normalizar los datos anidados en 'items' y desanidarlos en un nuevo DataFrame
df_items_desanidado = pd.json_normalize(df_items_exploded['items'])

# Reindexar los DataFrames
df_items_exploded.reset_index(inplace=True)
df_items_desanidado.reset_index(inplace=True)

# Concatenar los DataFrames y eliminar la columna 'items' que previamente desanidamos
user_items = pd.concat([df_items_exploded, df_items_desanidado], axis=1)
df_items = user_items.drop(columns=['items'])

#** df_items ahora contiene los datos desanidados**
# Muestra el DataFrame resultante 
df_items

,index,user_id,items_count,steam_id,user_url,index,item_id,item_name,playtime_forever,playtime_2weeks
0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,0,10,Counter-Strike,6.0,0.0
1,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1,20,Team Fortress Classic,0.0,0.0
2,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,2,30,Day of Defeat,7.0,0.0
3,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,3,40,Deathmatch Classic,0.0,0.0
4,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,4,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
5170010,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170010,373330,All Is Dust,0.0,0.0
5170011,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170011,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170012,521570,You Have 10 Seconds 2,4.0,4.0
5170013,88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,5170013,519140,Minds Eyes,3.0,3.0


In [58]:
df_items.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 10 columns):
 #   Column            Dtype  
---  ------            -----  
 0   index             int64  
 1   user_id           object 
 2   items_count       int64  
 3   steam_id          object 
 4   user_url          object 
 5   index             int64  
 6   item_id           object 
 7   item_name         object 
 8   playtime_forever  float64
 9   playtime_2weeks   float64
dtypes: float64(2), int64(3), object(5)
memory usage: 394.4+ MB


In [59]:
# realizamos la verificacion de datos nulos dentro de nuestro DataFrame 
df_items.isnull().sum()

index                   0
user_id                 0
items_count             0
steam_id                0
user_url                0
index                   0
item_id             16806
item_name           16806
playtime_forever    16806
playtime_2weeks     16806
dtype: int64

In [60]:
df_items.dropna(inplace=True)

In [61]:
# verificamos si existen datos duplicados en nuestro DataFrame
df_items.duplicated().sum()

0

In [62]:
# cambiamos el tipo de dato de la columna 'item_id'
df_items['item_id'] = df_items['item_id'].astype(int)

In [63]:
df_items.drop(columns=['index','items_count','user_url','index','playtime_2weeks','steam_id'],inplace=True)

In [64]:
df_items

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6.0
1,76561197970982479,20,Team Fortress Classic,0.0
2,76561197970982479,30,Day of Defeat,7.0
3,76561197970982479,40,Deathmatch Classic,0.0
4,76561197970982479,50,Half-Life: Opposing Force,0.0
...,...,...,...,...
5170009,76561198329548331,346330,BrainBread 2,0.0
5170010,76561198329548331,373330,All Is Dust,0.0
5170011,76561198329548331,388490,One Way To Die: Steam Edition,3.0
5170012,76561198329548331,521570,You Have 10 Seconds 2,4.0


##### RESUMEN
        * trabajamos con cada uno de los datasets ('steam_games','user_reviews','user_items')
        * transfromamos las columnas necesarias para su correcto uso 
        * se eliminaron todos los datos nulos ya que la cantidad de datos existtentes aun es suficiente para un correcto analisis 
        * se eliminaron las columnas que no son necesarias para nuestro trabajo y poder optimizar nuestros recursos 
        * en el dataset 'user_reviews' se creo la columna 'sentiment_analysis'

 

### CARGA DE DATOS(LOAD)

In [71]:
df_steam_games.to_parquet('steam_games.parquet')
df_user_reviews.to_parquet('user_reviews.parquet')
df_items.to_parquet('users_items.parquet')

##### RESUMEN 
    * se cargaron los datasets en formato ".parquet" para utilizar menos espacio y mejorar los recursos al momento de utilizarlos 